In [7]:
import os
os.chdir('/home/jovyan/work/quant_methods/Assessment/Group Work/Data')
print(os.getcwd())


/home/jovyan/work/quant_methods/Assessment/Group Work/Data


In [9]:
# importing libraries
import pandas as pd # type: ignore

df_collisions = pd.read_csv('dft-road-casualty-statistics-collision-last-5-years.csv')
print(f"Data frame 1 is {df_collisions.shape[0]:,} x {df_collisions.shape[1]}")
df_vehicles = pd.read_csv('dft-road-casualty-statistics-vehicle-last-5-years.csv')
print(f"Data frame 2 is {df_vehicles.shape[0]:,} x {df_vehicles.shape[1]}")
df_persons = pd.read_csv('dft-road-casualty-statistics-casualty-last-5-years.csv')
print(f"Data frame 3 is {df_persons.shape[0]:,} x {df_persons.shape[1]}")

/tmp/ipykernel_1094/1646971792.py:4: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collisions = pd.read_csv('dft-road-casualty-statistics-collision-last-5-years.csv')


Data frame 1 is 520,084 x 37


/tmp/ipykernel_1094/1646971792.py:6: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicles = pd.read_csv('dft-road-casualty-statistics-vehicle-last-5-years.csv')


Data frame 2 is 953,559 x 34
Data frame 3 is 665,408 x 21


/tmp/ipykernel_1094/1646971792.py:8: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_persons = pd.read_csv('dft-road-casualty-statistics-casualty-last-5-years.csv')


In [12]:
# Merge df_collisions with df_vehicles on 'accident_index'
df_coll_veh = pd.merge(df_collisions, df_vehicles, on='accident_index', how='outer')

# Merge the resulting dataframe with df_persons on 'accident_index'
df_merged = pd.merge(df_coll_veh, df_persons, on='accident_index', how='outer')

print(df_merged.shape)

(1423431, 90)


In [19]:
df_vru = df_merged[(df_merged['police_force'].isin([1, 48])) &  # Metropolitan & City of London Police
                   (df_merged['casualty_type'].isin([0,1,2,3,4,5,22,23,97,103,104,105,106])) # VRUs
]

print(df_vru.shape)

(76518, 90)


In [21]:
#print(df_vru.sample(20))
value_counts = df_vru['casualty_type'].value_counts()
print(value_counts)

casualty_type
1.0     27836
3.0     22828
0.0     15561
5.0      4267
4.0      3590
2.0      1935
23.0      323
97.0       99
22.0       79
Name: count, dtype: int64


In [31]:
cols = ['accident_index','accident_year', 'accident_reference', 'longitude', 'latitude', 'police_force', 
        'accident_severity', 'date', 'day_of_week', 'local_authority_district', 'road_type', 
        'speed_limit', 'light_conditions', 'weather_conditions', 'road_surface_conditions', 
        'lsoa_of_accident_location', 'casualty_class', 'sex_of_casualty', 'casualty_severity', 
        'pedestrian_location', 'casualty_type', 'casualty_distance_banding','time']
        


In [32]:
df_vru_s1 = df_vru[cols]
print(df_vru_s1.shape)

(76518, 23)


In [ ]:
print(df_vru_s1.sample(20))

In [ ]:
df_vru_s1.isnull().sum(axis=0).sort_values(ascending=False)

In [35]:
# save to csv
df_vru_s1.to_csv('df_vru_s1.csv', index=False)
df_vru.to_csv('df_vru.csv', index=False)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Create a geometry column from 'latitude' and 'longitude' columns
geometry = [Point(xy) for xy in zip(df_vru_s1['longitude'], df_vru_s1['latitude'])]

# Create a GeoDataFrame
gdf_vru_s1 = gpd.GeoDataFrame(df_vru_s1, geometry=geometry)

print(gdf_vru_s1.head())

In [37]:
gdf_vru_s1.to_file('gdf_vru_s1.gpkg', layer='vru_s1', driver='GPKG')

/opt/conda/lib/python3.11/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
